In [1]:
from Selector import *
import sys
import time
from datetime import datetime
from view.HTML import show
import json
from indicators.MA import *
# from indicators.Trend import *
from strategy.sequence.MAStrategy import *
from strategy.sequence.RStrategy import *
from simulation.Simulator import *
sys.path.append("../data")
import KDataDao as kds
from view.ViewDataFactory import *
from strategy.sequence.StrategyS1 import *
from simulation.Simulator import *

In [2]:

def decide(list,i,trend):
        """
        策略
        :param list: 测试数据
        :param trader: 交易者
        :param i: 当前在list中的位置
        :return:
        """
        k = list[i]
        pre = list[i-1]
        lm1 = trend.getMerge(-1)
        lm2 = trend.getMerge(-2)
        lm3 = trend.getMerge(-3)
        lm4 = trend.getMerge(-4)
        lm5 = trend.getMerge(-5)
        lm6 = trend.getMerge(-6)
        lm7 = trend.getMerge(-7)
        lm8 = trend.getMerge(-8)
        ls1 = trend.getSegment(-1)
        ls2 = trend.getSegment(-2)
        ls3 = trend.getSegment(-3)
        ls4 = trend.getSegment(-4)
        ls5 = trend.getSegment(-5)
        ls6 = trend.getSegment(-6)
        ls7 = trend.getSegment(-7)
        ls8 = trend.getSegment(-8)
        lb1 = trend.getBig(-1)
        lb2 = trend.getBig(-2)
        lb3 = trend.getBig(-3)
        lb4 = trend.getBig(-4)
        lb5 = trend.getBig(-5)
#      
        #买策略
        if(
            (
                (
                    (
                    lm2.ratio < -0.35 or lm2.day > 20) and
                    ls2.absR*2 < lm2.absR and
                    ls2.low < ls3.low and
                    (abs(ls3.absR-ls4.absR) < 0.02 or
                     abs(ls4.absR-ls5.absR) < 0.02 or
                     (min(ls3.low,ls5.low) > ls2.middle and  max(ls3.high,ls5.high) < lm2.middle ) or
                     ls3.day > 3 or ls3.ratio > 0.05) and
                    ls2.ratio < -0.05 and lm2.low < lm3.low
                )
                or
                (
                    lm6.low > lm4.low and
                    lm2.ratio < -0.05 and
                    lm2.absR*1.5 < lm4.absR and
                    (lm2.low - lm4.low)/lm4.low < -0.04 
                )
                or
                (
                    lm4.ratio < -0.2 and
                    lm2.ratio < -0.05 and
                    lm2.absR*1.5 < lm4.absR and
                    lm2.low < lm4.low
                )
                or(
                    lm4.day > 20 and
                    lm4.ratio < -0.1 and
                    lm2.ratio < -0.05 and
                    lm2.absR*1.5 < lm4.absR and
                    lm2.low < lm4.low
                )
            )
                and k.ratio() > 0.01 and lm1.day < 3

        ):
        # \
        #  \_
        #    \
#             print "%s,%s,%s,%s" % (list[0].code,datetime.fromtimestamp(ls1.ek.date/1000.0-24*60*60),ls1.isUp(),ls1.ek.close)
            return True,1
        if(
            lb2.ratio < -0.3 and
            lm2.absR*2 < lb2.absR and
            (lm2.ratio < -0.1 or lm2.day > 5) and
            k.ratio() > 0.01 and lm1.day < 3

        ):
        # \
        #  \_
        #    \
            return True,2
        if(
            lm6.ratio < -0.1 and
            lm2.absR < lm6.absR and
            (lm2.low - lm6.low)/lm6.low < -0.02 and
            (lm2.low - lm4.low)/lm4.low < -0.02 and
            lm2.ratio < -0.05 and
            lm6.middle > max(lm5.high,lm2.high) and
            lm1.day < 3
            and k.ratio() > 0.01
        ):
        #\
        # \
        #  \/\/\
        #       \
            return True,3
        if(
            lm6.ratio < -0.1 and
            ls2.absR*2 < lm2.absR and
            lm2.middle < max(lm5.low,lm2.low) and
            lm6.middle > max(lm5.high,lm2.high) and
            lm1.day < 3
            and k.ratio() > 0.01
        ):
        # \
        #  \/\/\
        #       \
            return True,4
        if(
            lm2.low < lm3.middle and
            lm3.high < lm4.middle and 
            ((lm4.absR > 0.1 and lm4.day > 5) or lm4.absR > 0.2) and
            (lm3.ratio > 0.1 or lm3.day > 7) and
            lm2.low > lm4.low and
            lm2.ratio < -0.04 and
            lm1.day < 3 and
            k.ratio() > 0.01 and
            lm2.absR*1.3 < lm4.absR and
            lm2.slope > -0.045
        ):
        #\
        # \/\,
            return True,5
        if(
            lm2.low > lm4.low and
            lm4.absR*2 < lm6.absR and
            lm6.low > lm5.middle and
            lm4.low < lm6.low and
            lm1.day < 3 
#                 and
#                 (k.ratio() > 0.02 or (k.close - pre.close)/pre.close > 0.02)
        ):
        #\
        # \
        #  \/\ 
        #     \/\,
            return True,6

        if k.isUp():
            if(
                abs((lm2.low - lm6.low)/lm6.low) < 0.04 and
                lm3.low > lm6.low and
                lm2.low > lm6.low and
                max(lm2.absR,lm5.absR)*2 < lm6.absR and
                lm1.day < 3 and max(lm2.low,lm6.low) < lm7.middle and
                lm2.isDown()
            ):
                #\
                # \   /\/\
                #  \/     \,
                return True,7
            if(
                lm6.ratio < -0.04 and 
                abs( max(lm5.high,lm2.high) - lm6.middle )/lm2.middle < 0.05 and
                lm2.low > lm6.low and lm3.low > lm6.low and
                lm2.ratio < -0.1 and lm2.slope < -0.03 and
                lm2.isDown()
            ):
                #\
                # \   /\/\
                #  \/     \,
                return True,8
            if(
                (
                    (
                        lm1.ratio < -0.25 and
                        (
                            (lm1.slope < -0.04 and lm1.mergeNum > 1) or 
                            (lm1.ratio < -0.4 and ls1.slope < -0.04) 
                        )
                        and k.ratio() > 0.02
                    ) 
                    or
                    (
                        lm2.ratio < -0.25 and
                        lm1.up < 3 and
                        (
                            (lm2.slope < -0.04 and lm2.mergeNum > 1 and k.ratio() > 0.02) or 
                            (lm2.ratio < -0.4 and lm2.slope < -0.02 and not k.isDown()) 
                        )
                        and lm1.ratio < 0.1
                    )
                ) 

            ):
                #\
                # \
                #  \,
                return True,9
        return False,-1

In [3]:
def firgueF(self,code):
    """显示股票"""
    #获取数据
    begin = "2016-01-05"
    end = None
    begin = "2015-01-05"
#     end = '2016-07-22'
    listK = kd.get_k_data(code,begin ,end)
    data,volume = ViewDataFactory.getCandlestick(listK)
    madic = ViewDataFactory.getMA(listK,(20,60))
    trendLine,trendMergeLine,trendBig = ViewDataFactory.getTrendLine(listK)
    
    #获取开始显示的时间
    mDate = 0
    if len(data)>0:
        if len(listK)/2>abs(-250):
            mDate = data[-250][0]
        else:
            mDate = data[len(listK)/2][0]
            
    #显示
    params = {}
    params["code"] = code
    params["condition"] = self.condition[code]
    params["mDate"] = mDate
    params["data"] = json.dumps(data)
    params["volume"] = json.dumps(volume)
    params["line20"] = json.dumps(madic[20].line)
    params["line60"] = json.dumps(madic[60].line)
    params["trendLine"] = json.dumps(trendLine)
    params["trendMergeLine"] = json.dumps(trendMergeLine)
    params["trendBig"] = json.dumps(trendBig)
    clear_output()
    show("Selector.html",params)


In [4]:
class KB(Selector):
    """红绿红"""
    condition = {}
    def analyze(self,list):
        """分析股票"""
        if len(list) < 200:
            return False
        i = 0
        flag = False
        trend = Trend()
        while i < len(list):
            trend.add(list[i])
            i += 1
        flag,condition = decide(list,len(list)-1,trend)
        if flag == True:
            self.condition[list[0].code] = condition
        return flag
    
    def firgue(self,code):
        """显示股票"""
        firgueF(self,code)

In [5]:
kb = KB()
